**Item requerido:** Utilizando plataforma embarcada:
- https://studio.edgeimpulse.com/
- Device: Celular
- Avaliar modelo gerado.
- Gerar Notebook Python com teste do modelo.

## Projeto desenvolvido no Edge Impulse

Link de acesso do projeto: [rwri/doencaMIlho](https://studio.edgeimpulse.com/public/410033/live)

## Bibliotecas


In [ ]:
# Instalar bibliotecas necessárias
!pip install tensorflow opencv-python

# Importar bibliotecas necessárias
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Caregando o modelo

In [ ]:
# Carregar o modelo TensorFlow Lite
interpreter = tf.lite.Interpreter(model_path="modeloDoencaMilhoRGB.tflite")
interpreter.allocate_tensors()

# Obter detalhes da entrada e saída do modelo
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

## Baixando base de dados

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download 'smaranjitghose/corn-or-maize-leaf-disease-dataset'
!unzip -qq corn-or-maize-leaf-disease-dataset.zip
!rm corn-or-maize-leaf-disease-dataset.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/smaranjitghose/corn-or-maize-leaf-disease-dataset
License(s): copyright-authors
 98% 157M/161M [00:02<00:00, 46.6MB/s]
100% 161M/161M [00:02<00:00, 56.6MB/s]


## Pré-processamento e carregamento de imagens aleatórias

In [ ]:
def preprocess_image(image_path, input_shape):
    # Carregar a imagem
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Redimensionar a imagem
    img_resized = cv2.resize(img, (input_shape[1], input_shape[2]))

    # Normalizar a imagem
    img_normalized = img_resized.astype('float32') / 255.0

    # Expandir as dimensões para incluir o batch size
    img_expanded = np.expand_dims(img_normalized, axis=0)

    return img_expanded

import os
import random

def get_random_images(data_dir, num_images):
    # Lista para armazenar os caminhos das imagens
    image_paths = []

    # Obter a lista de subpastas
    subfolders = [f.path for f in os.scandir(data_dir) if f.is_dir()]

    # Para cada subpasta
    for subfolder in subfolders:
        # Obter a lista de imagens na subpasta
        images = [f.path for f in os.scandir(subfolder) if f.is_file() and f.name.endswith(('.png', '.jpg', '.jpeg'))]

        # Selecionar imagens aleatórias
        random_images = random.sample(images, num_images)

        # Adicionar os caminhos das imagens à lista
        image_paths.extend(random_images)

    return image_paths

# Usar a função para obter um vetor com imagens aleatórias
data_dir = 'data'
num_images = 5  # Número de imagens aleatórias para selecionar de cada subpasta
random_image_paths = get_random_images(data_dir, num_images)
input_shape = input_details[0]['shape']

# Pré-processar as imagens
preprocessed_images = [preprocess_image(image_path, input_shape) for image_path in random_image_paths]

## Inferência

In [ ]:
import cv2

# Definir o tamanho desejado
desired_size = (224, 224)

# Definir as classes
classes = ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

# Criar uma nova figura
plt.figure(figsize=(20, 20))

# Para cada imagem pré-processada
for i, preprocessed_image in enumerate(preprocessed_images):
    # Definir os dados de entrada do modelo
    interpreter.set_tensor(input_details[0]['index'], preprocessed_image)

    # Executar a inferência
    interpreter.invoke()

    # Obter os resultados da inferência
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Interpretar os resultados
    predicted_class = np.argmax(output_data[0])
    predicted_class_name = classes[predicted_class]
    confidence = output_data[0][predicted_class]

    # Ler e redimensionar a imagem
    image = cv2.imread(random_image_paths[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, desired_size)

    # Exibir a imagem em um subplot
    plt.subplot(5, 4, i+1)
    plt.imshow(image)
    plt.title(f"Imagem: {os.path.basename(random_image_paths[i])}\nClasse prevista: {predicted_class_name}\nConfiança de: {confidence:.2f}")
    plt.axis('off')

# Mostrar todos os subplots
plt.tight_layout()
plt.show()

## Extra - Escala de Cinza

In [ ]:
# Carregar o modelo TensorFlow Lite
interpreter = tf.lite.Interpreter(model_path="modeloDoencaMilhoGray.tflite")
interpreter.allocate_tensors()

# Obter detalhes da entrada e saída do modelo
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def preprocess_image_gray(image_path, input_shape):
    # Carregar a imagem em escala de cinza
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Redimensionar a imagem
    img_resized = cv2.resize(img, (input_shape[1], input_shape[2]))

    # Normalizar a imagem
    img_normalized = img_resized.astype('float32') / 255.0

    # Expandir as dimensões para incluir o batch size e o canal de cor
    img_expanded = np.expand_dims(img_normalized, axis=0)
    img_expanded = np.expand_dims(img_expanded, axis=-1)

    return img_expanded

# Pré-processar as imagens
preprocessed_images = [preprocess_image_gray(image_path, input_shape) for image_path in random_image_paths]

import cv2

# Definir o tamanho desejado
desired_size = (224, 224)

# Definir as classes
classes = ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

# Criar uma nova figura
plt.figure(figsize=(20, 20))

# Para cada imagem pré-processada
for i, preprocessed_image in enumerate(preprocessed_images):
    # Definir os dados de entrada do modelo
    interpreter.set_tensor(input_details[0]['index'], preprocessed_image)

    # Executar a inferência
    interpreter.invoke()

    # Obter os resultados da inferência
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Interpretar os resultados
    predicted_class = np.argmax(output_data[0])
    predicted_class_name = classes[predicted_class]
    confidence = output_data[0][predicted_class]

    # Ler e redimensionar a imagem em escala de cinza
    image = cv2.imread(random_image_paths[i], cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, desired_size)

    # Exibir a imagem em um subplot
    plt.subplot(5, 4, i+1)
    plt.imshow(image, cmap='gray')
    plt.title(f"Imagem: {os.path.basename(random_image_paths[i])}\nClasse prevista: {predicted_class_name}\nConfiança de: {confidence:.2f}")
    plt.axis('off')

# Mostrar todos os subplots
plt.tight_layout()
plt.show()